# Environment installation

In [4]:
# Install packages
# %pip install -U scikit-learn
%pip install ftfy
%pip install optuna
%pip install darts

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [42]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import joblib
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from darts import TimeSeries
from darts.models import KalmanForecaster, RandomForest, LinearRegressionModel, LightGBMModel, \
                        CatBoostModel, RNNModel, BlockRNNModel, NBEATSModel, NHiTSModel, \
                        TCNModel, TransformerModel, TFTModel
from darts.metrics import mae
from darts.dataprocessing.transformers import Scaler

from darts.utils.likelihood_models import GaussianLikelihood

from ftfy import fix_text
import warnings
warnings.filterwarnings('ignore')

# from google.colab import drive

In [ ]:
# prj_path = '/home/mlworker/Quang/HealthCare/Source_14012023_v4/'
# data_path = prj_path + "data/new_data/DH/squeezed/"
# prj_path_opt= prj_path + "optimize_hyperparam/opt_results/LinearRegression_27022023/"

# os.chdir(prj_path)

# Configuration

In [7]:
# !pwd
# /Source_14012023_v4/preprocessing_data

In [72]:
prj_path = '../'
data_path = prj_path + "data/new_data/DH/squeezed/"
prj_path_opt= prj_path + "optimize_hyperparam/opt_results/"
output_process = prj_path + "data/new_data/DH/processed_data/"

In [47]:
data_set = {0 : "train", 1 : "test"}

In [11]:
all_cities = [
        'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
        'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
        'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh',
        'Hòa Bình','Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
        'Lạng Sơn','Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
        'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
        'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
        'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
        'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp'
]
cities = ['An Giang']

In [35]:
# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
      # lấy bộ test dài 36 tháng = 3 năm
        self.test_size = 36
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng        
        self.look_back = 3
        # dự phóng n-step trong 6 tháng
        self.n_predicted_period_months = 6
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        self.device = torch.device("cuda")
        self.epochs = 300

args = Configuration()

# Seeding

In [ ]:
def seed_everything(seed: int):
    import random, os
    import numpy as np
    import torch
    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(42)

# Supporting functions

In [13]:
def get_dict_all_city_data():
  cities_data = {}  
  for city in cities:
    city_result = pd.read_excel(prj_path+'data/new_data/DH/squeezed/squeezed_'+city+'.xlsx')  
    """Get all data from all city in 1997 - 2016""" 
    city_result = city_result.loc[city_result['year_month'] < '2017-1-1'] 
    cities_data[city] = city_result
  return cities_data

In [26]:
# Define data (pre-)processing functions
# modification
def get_city_data(city_name, dict_full_data):
    """Returns Diarrhoea rate and climate data""" 
    city_data = dict_full_data[city_name].drop(columns=['Diarrhoea_cases','Diarrhoea_rates', 'province',
                                                        'Influenza_rates','Influenza_cases',
                                                        'Dengue_fever_cases', 'year', 'month'], 
                                                                  axis=1, 
                                                                  inplace=False)    
    return city_data

def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Imputes 0 for first 12 months, 
    last year's value for months 12-24, 
    and minimum value of last two years for months 25+
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

In [30]:
def clean_full_data(dict_full_data):
    climate_and_disease_feats = ['Total_Evaporation',
       'Total_Rainfall', 'Max_Daily_Rainfall', 'n_raining_days',
       'Average_temperature', 'Max_Average_Temperature',
       'Min_Average_Temperature', 'Max_Absolute_Temperature',
       'Min_Absolute_Temperature', 'Average_Humidity', 'Min_Humidity',
       'n_hours_sunshine', 'Dengue_fever_rates']
    for city in cities:
        city_data = get_city_data(city_name=city,dict_full_data = dict_full_data)
        city_data_features = city_data[climate_and_disease_feats]
        city_data_features = impute_missing_value(city_data_features)
        city_data_features = convert_to_stationary(city_data_features)
        city_data_features.dropna(inplace=True)
        city_data_features.loc[:, "year_month"] = city_data["year_month"]
        dict_full_data[city] = city_data_features
    return dict_full_data


In [32]:
def split_data(data, look_back ):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]    
    test = data[-args.test_size - look_back: ]
    return train, test

In [159]:
def to_supervised(data,  d_out, d_in, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[out_end-1: out_end, -1])
    return np.array(X), np.array(y)

In [153]:
def select_feature(train, next_predicted_month):
    """Selects args.n_features top features using RFE"""
    train_X, train_y = to_supervised(train, d_out=next_predicted_month, d_in=args.look_back )
    print("Shape train_X: ", train_X.shape)
    print("Shape train_y: ", train_y.shape)
    train_X, train_y = np.squeeze(train_X), np.squeeze(train_y)
    rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=args.seed), n_features_to_select=args.n_features)
    fit = rfe.fit(train_X, train_y)
    important_features = list()
    for i in range(len(fit.support_)):
        if fit.support_[i]:
            important_features.append(i)
    return np.array(important_features)

In [160]:
train_X, train_y = to_supervised(train, d_out=next_predicted_month, d_in=args.look_back )

In [ ]:
def featureSelection(Xtrain, Ytrain, n_selection_feature, n_selection_feature_by_day = 18):
  D = Xtrain.shape[2]
  
  rfe = RFE(RandomForestRegressor(), n_features_to_select=n_selection_feature_by_day)

  fit = rfe.fit(Xtrain.reshape(len(Xtrain),D*args.look_back), Ytrain)
  important_features = list()
  for i in range(len(fit.support_)):
      if fit.support_[i]:
          important_features.append(i)
  result = np.array(important_features)
  calMostFeature = [0]*D
  for i in result:
    calMostFeature[i%D] = calMostFeature[i%D] + 1
  top_idx =  np.sort(np.argsort(calMostFeature)[-n_selection_feature:])
  return top_idx

In [145]:
def get_data(train_np, test_np, batch_size, specific_data):
    """
    Returns important feature list and data formatted for input into Pytorch 
    models
    """
    important_features = select_feature(train_np, specific_data)

    train_X, train_y = to_supervised(train_np, features_list=important_features)

    test_X, test_y = to_supervised(test_np, features_list=important_features)
    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    return important_features, train_loader, test_tensor

In [108]:
def normalizationMinMax(df,city,data_set_index):
    date_index = df.index
    norm_set = data_set[data_set_index]
    scaler = MinMaxScaler()
    if norm_set == "train":
        scaler.fit(df)
        series = scaler.transform(df)
        df_scaled = pd.DataFrame(data = series, columns = df.columns)
        joblib.dump(scaler, output_process+city+'_train_scalerMinMaxNorm.save')

    else:
        true_incidence = df.iloc[:, [-1]]
        scaler = joblib.load(output_process+city+'_train_scalerMinMaxNorm.save') #ok
        series = scaler.transform(df)
        df_scaled = pd.DataFrame(data = series, columns = df.columns)
        df_scaled.iloc[:, [-1]] = true_incidence
    df_scaled["year_month"] = date_index
    """Save data as csv, when load data as a dataframe, use command df.iloc[:,:-1].to_numpy() to convert to an array to use"""
    df_scaled.to_csv(output_process+city+'_'+norm_set+'_preprocessed.csv', float_format='%.7f',index=False)
    return df_scaled,scaler

# Run processing data

In [109]:
dict_full_data = get_dict_all_city_data()
full_data = clean_full_data(dict_full_data=dict_full_data)
for city in cities:
    specific_data = full_data[city]
    specific_data = specific_data.set_index("year_month")
    train, test = split_data(specific_data, args.look_back)
    df_train,scaler = normalizationMinMax(train,city, 0)
    df_test,scaler = normalizationMinMax(test,city, 1)

# Run feature selection

In [154]:
dict_full_data = get_dict_all_city_data()
full_data = clean_full_data(dict_full_data=dict_full_data)
for city in cities:
    df = pd.read_csv(output_process+city+'_train_preprocessed.csv', parse_dates=True, index_col= None, encoding = 'unicode_escape')
    train = df.iloc[:,:-1].to_numpy()
    next_predicted_month = 2
    important_features = select_feature(train, next_predicted_month)
    print(important_features)
    list_features = full_data[city].columns.tolist()
    selected_features = [
        list_features[important_features[0]],
        list_features[important_features[1]],
        list_features[important_features[2]]
    ]
    print(selected_features)

Shape train_X:  (199, 3, 13)
Shape train_y:  (199, 2)


ValueError: Found array with dim 3. RFE expected <= 2.

In [141]:
next_predicted_month = 2
important_features = select_feature(train, next_predicted_month)
list_features = full_data[city].columns.tolist()
selected_features = [
    list_features[important_features[0]],
    list_features[important_features[1]],
    list_features[important_features[2]]
]

In [142]:
print(selected_features)

['n_raining_days', 'Min_Absolute_Temperature', 'Dengue_fever_rates']


In [139]:
def select_feature(train, next_predicted_month):
    """Selects args.n_features top features using RFE"""
    train_X, train_y = to_supervised(train, d_out=next_predicted_month, d_in=1 )
    train_X, train_y = np.squeeze(train_X), np.squeeze(train_y)
    rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=args.seed), n_features_to_select=args.n_features)
    fit = rfe.fit(train_X, train_y)
    important_features = list()
    for i in range(len(fit.support_)):
        if fit.support_[i]:
            important_features.append(i)
    return np.array(important_features)

In [ ]:
def select_feature(train, next_predicted_month):
    """Selects args.n_features top features using RFE"""
    train_X, train_y = to_supervised(train, d_in=1, d_out=1)
    train_X, train_y = np.squeeze(train_X), np.squeeze(train_y)
    rfe = RFE(RandomForestRegressor(n_estimators=500, random_state=args.seed), n_features_to_select=args.n_features)
    fit = rfe.fit(train_X, train_y)
    important_features = list()
    for i in range(len(fit.support_)):
        if fit.support_[i]:
            important_features.append(i)
    return np.array(important_features)

In [126]:
# norm
scaler = MinMaxScaler(feature_range=(-1, 1))
list_features = full_data[city].columns.tolist()
labels = ["Dengue_fever_rates"]
df_features = full_data[city]

# SELECT FETURES
specific_data = full_data[city]
specific_data = specific_data.set_index("year_month")

train, test = split_data(specific_data, args.look_back)

# Fit data scaler to training data
full_scaler = MinMaxScaler().fit(train)
y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

# Scale train and test data
train = full_scaler.transform(train)
test = full_scaler.transform(test)

# Get data to run model
important_features = select_feature(train, specific_data)

# selected_features = [
#     list_features[important_features[0]],
#     list_features[important_features[1]]
# ]

In [129]:
important_features

array([ 2,  4, 12])

In [100]:
df_train.iloc[:,:-1].to_numpy().shape

array([[0.36724138, 0.46628065, 0.42813589, ..., 0.57894737, 0.203     ,
        0.51903695],
       [0.56954023, 0.46280654, 0.41811847, ..., 0.36842105, 0.9885    ,
        0.51402799],
       [0.52011494, 0.49645777, 0.50043554, ..., 0.23684211, 0.28      ,
        0.5530263 ],
       ...,
       [0.46551724, 0.47479564, 0.47560976, ..., 0.39473684, 0.83      ,
        0.53487389],
       [0.5       , 0.35490463, 0.24477352, ..., 0.39473684, 0.65      ,
        0.56060385],
       [0.57471264, 0.45367847, 0.52351916, ..., 0.55263158, 0.61      ,
        0.53153234]])

In [101]:
train

array([[0.36724138, 0.46628065, 0.42813589, ..., 0.57894737, 0.203     ,
        0.51903695],
       [0.56954023, 0.46280654, 0.41811847, ..., 0.36842105, 0.9885    ,
        0.51402799],
       [0.52011494, 0.49645777, 0.50043554, ..., 0.23684211, 0.28      ,
        0.5530263 ],
       ...,
       [0.46551724, 0.47479564, 0.47560976, ..., 0.39473684, 0.83      ,
        0.53487389],
       [0.5       , 0.35490463, 0.24477352, ..., 0.39473684, 0.65      ,
        0.56060385],
       [0.57471264, 0.45367847, 0.52351916, ..., 0.55263158, 0.61      ,
        0.53153234]])

In [ ]:
def objective(model_name, trial, city):   
    scaler = MinMaxScaler(feature_range=(-1, 1))
    # ==================================================================
    list_features = full_data[city].columns.tolist()
    labels = ["Diarrhoea_rates"]
    df_features = full_data[city]

    # SELECT FETURES
    specific_data = full_data[city]
    specific_data = specific_data.set_index("year_month")

    train, test = split_data(specific_data, args.look_back)

    # Fit data scaler to training data
    full_scaler = MinMaxScaler().fit(train)
    y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

    # Scale train and test data
    train = full_scaler.transform(train)
    test = full_scaler.transform(test)

    # Get data to run model
    important_features = select_feature(train, specific_data)

    selected_features = [
        list_features[important_features[0]],
        list_features[important_features[1]]
    ]

    if model_name == "RandomForest":
      random_state = trial.suggest_int('random_state', 0, 42)
      n_estimators = trial.suggest_int('n_estimators', 50, 200)
      max_depth = trial.suggest_int('max_depth', 1, 15)
      output_chunk_length = trial.suggest_int('output_chunk_length', 1, 3)
      # Create the RandomForest model
      model = RandomForest(
                    lags = 3,
                    lags_past_covariates = 3,
                    output_chunk_length = output_chunk_length,
                    n_estimators = n_estimators,
                    max_depth = max_depth,
                    random_state=random_state)
    elif model_name == 'TFTModel':
      # Define the hyperparameters to optimize
      random_state = trial.suggest_int('random_state', 0, 42)
      dropout = trial.suggest_uniform('dropout', 0.01, 0.8)
      n_epochs = trial.suggest_int('n_epochs', 50, 200)

      # Create the TFTModel model
      model = TFTModel(
                    input_chunk_length = 3,
                    output_chunk_length = 1,
                    add_relative_index = True,
                    dropout = dropout,
                    n_epochs = n_epochs ,
                    random_state=random_state)
    elif model_name == 'NHiTSModel':
      #suggest hyperparams
      random_state = trial.suggest_int('random_state', 0, 42)
      dropout = trial.suggest_uniform('dropout', 0.01, 0.80)
      n_epochs = trial.suggest_int('n_epochs', 100, 500, step=10)
      MaxPool1d = trial.suggest_categorical('MaxPool1d', [True, False])

      model = NHiTSModel(
                          input_chunk_length = 3,
                          output_chunk_length = 1,
                          MaxPool1d = MaxPool1d,
                          dropout = dropout,
                          n_epochs = n_epochs ,
                          random_state=random_state)
    elif model_name == 'LinearRegressionModel':
      random_state = trial.suggest_int('random_state', 0, 43)
      output_chunk_length = trial.suggest_int('output_chunk_length', 1, 5)
      
      # Create the  model
      model = LinearRegressionModel(
                      lags = output_chunk_length,
                      lags_past_covariates = 3,
                      output_chunk_length = output_chunk_length,
                      random_state=random_state)
    elif model_name == "BlockRNNModel":
      #suggest hyperparams
      random_state = trial.suggest_int('random_state', 0, 1000)
      n_rnn_layers = trial.suggest_int('n_rnn_layers', 1, 3)
      dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
      hidden_dim = trial.suggest_int('n_rnn_layers', 5, 20)
      n_epochs = trial.suggest_int('n_epochs', 50, 200)

      model = BlockRNNModel(
                          input_chunk_length = 3,
                          output_chunk_length = 1,
                          hidden_dim = hidden_dim,
                          n_rnn_layers = n_rnn_layers,
                          dropout = dropout,
                          n_epochs = n_epochs,
                          random_state=random_state)
      
    elif model_name == "CatBoostModel":
      #suggest hyperparams
      learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
      n_estimators = trial.suggest_int('n_estimators', 50, 200)
      max_depth = trial.suggest_int('max_depth', 1, 15)
      random_state = trial.suggest_int('random_state', 0, 1000)
      likelihood = trial.suggest_categorical('likelihood', ['quantile'])
      quantiles =  trial.suggest_categorical('quantiles', [None, [0.1, 0.5, 0.9]])
      output_chunk_length = trial.suggest_int('output_chunk_length', 1, 4)
      bagging_temperature = trial.suggest_float('bagging_temperature', 0.01, 100.0)
      border_count = trial.suggest_int('border_count', 1, 255)
      l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 0.1, 10)
      random_strength = trial.suggest_float('random_strength', 0.1, 10)
      model = CatBoostModel(lags=3,
                          lags_past_covariates=3, 
                          learning_rate=learning_rate,
                          n_estimators=n_estimators,
                          max_depth=max_depth, 
                          output_chunk_length = output_chunk_length,
                          likelihood = likelihood,
                          quantiles = quantiles,
                          bagging_temperature = bagging_temperature,
                          border_count = border_count,
                          l2_leaf_reg = l2_leaf_reg,
                          random_strength = random_strength,
                          random_state=random_state)
    
    elif model_name == "NBEATSModel":
      random_state = trial.suggest_int('random_state', 0, 42)
      dropout = trial.suggest_uniform('dropout', 0.01, 0.80)
      n_epochs = trial.suggest_int('n_epochs', 50, 200)

      pl_trainer_kwargs = {
              "accelerator": "gpu",
              "devices": -1,
              "auto_select_gpus": True,
          }
      model = NBEATSModel(
                            input_chunk_length = 3,
                            output_chunk_length = 1,
                            dropout = dropout,
                            n_epochs = n_epochs ,
                            pl_trainer_kwargs = pl_trainer_kwargs,
                            random_state=random_state)
  
    elif model_name == "TCNModel":
      params = {
        'kernel_size': trial.suggest_int("kernel_size", 2, 5),
        'num_filters': trial.suggest_int("num_filters", 1, 5),
        'weight_norm': trial.suggest_categorical("weight_norm", [False, True]),
        'dilation_base': trial.suggest_int("dilation_base", 2, 4),
        'dropout': trial.suggest_float("dropout", 0.0, 0.4),
        'learning_rate': trial.suggest_float("learning_rate", 5e-5, 1e-3, log=True),
        'include_year': trial.suggest_categorical("year", [False, True]),
        'n_epochs': trial.suggest_int("n_epochs", 100, 300),
      }

      # select input and output chunk lengths
      in_len = trial.suggest_int("input_chunk_length", 12, 36)
      out_len = trial.suggest_int("output_chunk_length", 1, in_len-1)

      params['input_chunk_length'] = in_len
      params['lags'] = in_len
      params['output_chunk_length'] = out_len    
   

      # optionally also add the (scaled) year value as a past covariate
      if params['include_year']:
          encoders = {"datetime_attribute": {"past": ["year"]},
                      "transformer": Scaler()}
      else:
          encoders = None
      params['encoders'] = encoders

      pl_trainer_kwargs = {
            "accelerator": "gpu",
            "devices": -1,
            "auto_select_gpus": True,
        }
     

      param = params
      model = TCNModel(
          input_chunk_length=param['input_chunk_length'],
          output_chunk_length=param['output_chunk_length'],
          batch_size=16,
          n_epochs=param['n_epochs'],
          nr_epochs_val_period=1,
          kernel_size=param['kernel_size'],
          num_filters=param['num_filters'],
          weight_norm=param['weight_norm'],
          dilation_base=param['dilation_base'],
          dropout=param['dropout'],
          optimizer_kwargs={"lr": param['learning_rate']},
          add_encoders=param['encoders'],
          likelihood=GaussianLikelihood(),
          pl_trainer_kwargs=pl_trainer_kwargs,
          model_name="tcn_model",
          force_reset=True,
          save_checkpoints=True,
      )
    elif model_name == "LightGBMModel":
      params = {
        "lags": trial.suggest_categorical("lags", [3]),
        "lags_past_covariates": trial.suggest_categorical("lags_past_covariates", [3]),
        "random_state": trial.suggest_int("random_state", 0, 999),
        "multi_models": trial.suggest_categorical("multi_models", [True, False]),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.5),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.1, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'verbose': -1,
        'likelihood' : trial.suggest_categorical("likelihood", ["quantile"])
      }

      param = params
      model = LightGBMModel(
          lags = param['lags'],
          lags_past_covariates = param['lags_past_covariates'],
          output_chunk_length = 1,
          random_state = param['random_state'],
          multi_models = param['multi_models'],
          likelihood = param['likelihood'],
          num_leaves = param['num_leaves'],
          learning_rate = param['learning_rate'],
          feature_fraction = param['feature_fraction'],
          bagging_fraction = param['bagging_fraction'],
          min_child_samples = param['min_child_samples'],
          lambda_l1 = param['lambda_l1'],
          verbose = param['verbose']
      )

    mae_error = output_prediction_for_location(df_features, model, location=city, feature_list=selected_features,
                                                       labels=labels, scaler=scaler, lags= 3 if param.get('input_chunk_length') is None else param['input_chunk_length'],
                                                       split_index=-36)

    return mae_error

# Main run optimize and save

In [ ]:
#########################
# Main Cell for optimize
#########################

# Input param for Optimize Run
ntry = 50
njob = 1

model_name_list = [
     "RandomForest",
     "LinearRegressionModel",
     "LightGBMModel",
     "CatBoostModel",
     "BlockRNNModel",
     "NBEATSModel",
     "NHiTSModel",
     "TCNModel",
     "TFTModel"
]

# Lưu thông tin traceback study và error city trong quá trình optimize
l_study_city ={}
l_errCity =[]

if __name__ == '__main__':  
  for model_name in model_name_list: 
    print(model_name)
    best_param = pd.DataFrame()
    for city in cities:
      # Use Tree-structured Parzen Estimator sampler to minimise RMSE
      sampler = optuna.samplers.TPESampler()
      study = optuna.create_study(sampler=sampler, direction='minimize', study_name = model_name)

      # truyền multiple param vào trong biến trial
      obj_func = lambda trial: objective(model_name, trial, city)

      try:
        # Optimise over 100 trials
        study.optimize(obj_func, n_trials=ntry, n_jobs=njob)

        # Print results
        print("Study statistics for : ")
        print("  Number of finished trials: ", len(study.trials))
    
        
        print("Best trial of city: ",city)
        best_trial = study.best_trial
        print("  Value: ", best_trial.value)   

        # lưu best param vào trong biến toàn cục

        if model_name == "LinearRegressionModel": 
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'LinearRegressionModel',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'lags' : best_trial.params['output_chunk_length'],
                              'lags_past_covariates': 3,
                              'output_chunk_length': best_trial.params['output_chunk_length'],
                              'random_state':best_trial.params['random_state'],
                              }, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')

        elif model_name == "LightGBMModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'LightGBMModel',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'lags': best_trial.params['lags'],
                              'lags_past_covariates': best_trial.params['lags_past_covariates'],
                              'multi_models': best_trial.params['multi_models'],
                              'num_leaves': best_trial.params['num_leaves'], 
                              'feature_fraction': best_trial.params['feature_fraction'], 
                              'min_child_samples': best_trial.params['min_child_samples'], 
                              'lambda_l1': best_trial.params['lambda_l1'], 
                              'lambda_l2': best_trial.params['lambda_l2'], 
                              'likelihood': best_trial.params['likelihood'], 
                              'learning_rate': best_trial.params['learning_rate']}, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')
           
        elif model_name == "CatBoostModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'CatBoost',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'lags' : 3,
                              'lags_past_covariates': 3,
                              'output_chunk_length': best_trial.params['output_chunk_length'],
                              'likelihood': best_trial.params['likelihood'],
                              'learning_rate': best_trial.params['learning_rate'],
                              'n_estimators': best_trial.params['n_estimators'],
                              'max_depth': best_trial.params['max_depth'],
                              'bagging_temperature': best_trial.params['bagging_temperature'],
                              'l2_leaf_reg': best_trial.params['l2_leaf_reg'],
                              'random_strength':best_trial.params['random_strength'],
                              }, index=[0])

          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')
          
        elif model_name == "NHiTSModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'N-HiTS',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'input_chunk_length' : 3,
                              'output_chunk_length' : 1,
                              'MaxPool1d' : best_trial.params['MaxPool1d'],
                              'dropout' : best_trial.params['dropout'],
                              'n_epochs' : best_trial.params['n_epochs'],
                              'random_state' : best_trial.params['random_state'],
                              }, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')

        elif model_name == "TCNModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'TCNModel',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'lags' : best_trial.params['input_chunk_length'],
                              'input_chunk_length': best_trial.params['input_chunk_length'],
                              'output_chunk_length': best_trial.params['output_chunk_length'],
                              'n_epochs':best_trial.params['n_epochs'],
                              'num_filters':best_trial.params['num_filters'],
                              'weight_norm':best_trial.params['weight_norm'],
                              'dilation_base':best_trial.params['dilation_base'],
                              'dropout':best_trial.params['dropout'],
                              'learning_rate':best_trial.params['learning_rate'],
                              'year':best_trial.params['year'],
                              }, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')
          
        elif model_name == "NBEATSModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'NBeatsModel',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'output_chunk_length': 3,
                              'input_chunk_length': 1,
                              'n_epochs':best_trial.params['n_epochs'],
                              'dropout':best_trial.params['dropout'],
                              'random_state':best_trial.params['random_state'],
                              }, index=[0])    
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')
        elif model_name == "TFTModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'TFTModel',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'output_chunk_length': 3,
                              'input_chunk_length': 1,
                              'add_relative_index': True,
                              'random_state':best_trial.params['random_state'],
                              'n_epochs':best_trial.params['n_epochs'],
                              'dropout':best_trial.params['dropout']
                              }, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')
        elif model_name == "RandomForest":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'RandomForest',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'lags' : best_trial.params['output_chunk_length'],
                              'lags_past_covariates': 3,
                              'output_chunk_length': best_trial.params['output_chunk_length'],
                              'n_estimators': best_trial.params['n_estimators'],
                              'max_depth': best_trial.params['max_depth'],
                              'random_state':best_trial.params['random_state'],
                              }, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')
           
        elif model_name == "BlockRNNModel":
          one_city_param = pd.DataFrame({
                              'City': city,
                              'Alg_name': 'BlockRNNModel',
                              'Best_value': best_trial.value,
                              'n_try_opt': ntry,
                              'input_chunk_length': 3,
                              'output_chunk_length': 1,
                              'random_state':best_trial.params['random_state'],
                              'n_epochs':best_trial.params['n_epochs'],
                              'hidden_dim': best_trial.params['hidden_dim'],
                              'n_rnn_layers': best_trial.params['n_rnn_layers'],
                              'dropout':best_trial.params['dropout']
                              }, index=[0])
          one_city_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + "_" +city+'.xlsx')

        best_param = best_param.append(one_city_param)
      except:# có error thì lưu vào l_errCity để check lại sau 
        l_errCity.append(city)
    # lưu kết quả vào file 
    best_param.to_excel(prj_path_opt+'diarrhoea_opt_hyperparam_' + model_name + '.xlsx')


In [ ]:
# import requests

# def send_to_telegram(message):

#     apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
#     chatID = '@ptn_announcement'
#     apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

#     try:
#         response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
#         print(response.text)
#     except Exception as e:
#         print(e)

# send_to_telegram("Server Chạy Xong optimize" )